In [3]:
import pandas as pd
import numpy as np
import folium
import time

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
# 웹드라이버 설정
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("headless")

# driver = webdriver.Chrome("../driver/chromedriver.exe", options=chrome_options)
driver = webdriver.Chrome()
driver.implicitly_wait(10)

# 스타벅스 지도 페이지 오픈
starbuck_map_url = "https://www.starbucks.co.kr/store/store_map.do?disp=locale"
driver.get(starbuck_map_url)

In [9]:
# 윈도우 화면 최대화
driver.maximize_window()

In [11]:
# 스타벅스 서울 클릭
starbucks_seoul_css = "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_css))).click()

In [13]:
# 스타벅스 서울 전체 클릭
starbucks_seoul_all_css = "#mCSB_2_container > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all_css))).click()

In [15]:
# 스타벅스 서울 전체 HTML 파싱
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [16]:
container = soup.find("div", id="mCSB_3_container")
li_all = container.find_all("li")

In [19]:
starbucks_data = []

for li in li_all:
    name = li.find("strong").text.strip()
    address = li.find("p").text.strip().replace("1522-3232", "")
    gu = address.split(" ")[1]    
    each = {
        "매장이름":name, "주소":address, "구":gu, "브랜드":"스타벅스"
    }
    starbucks_data.append(each)

In [21]:
len(starbucks_data)

634

In [23]:
df_starbucks = pd.DataFrame(starbucks_data)
df_starbucks.tail()

,매장이름,주소,구,브랜드
629,상봉역,서울특별시 중랑구 망우로 307 (상봉동),중랑구,스타벅스
630,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",중랑구,스타벅스
631,망우동,서울특별시 중랑구 망우로 407 (망우동),중랑구,스타벅스
632,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),중랑구,스타벅스
633,중화역,서울특별시 중랑구 봉화산로 35 1층,중랑구,스타벅스


In [25]:
gu_list = list(df_starbucks["구"].unique())
len(gu_list)

25

In [27]:
# 이디야 지도 페이지 오픈
eidya_map_url = "https://www.ediya.com/contents/find_store.html"
driver.get(eidya_map_url)

In [29]:
# 이디야 페이지 화면 최대화
driver.maximize_window()

In [31]:
# 이디야 주소 검색 클릭
eidya_address_css = "#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_address_css))).click()

In [33]:
eidya_data = []

# 이디야 지도 서울 > 지역별 데이터 검색
for gu in gu_list:

    eidya_search_keyword_css = "#keyword"
    
    # 이디야 주소 검색어 초기화
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_search_keyword_css))).clear()
    
    # 이디야 주소 검색어 입력
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_search_keyword_css))).send_keys(f"서울 {gu}")
    
    # 이디야 주소 검색 버튼 클릭
    eidya_search_button_css = "#keyword_div > form > button"
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_search_button_css))).click()
    
    # 이디야 구별 매장 정보 수집
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    ul_tag = soup.find("ul", id="placesList")
    dl_all = ul_tag.find_all("dl")
    
    for dl in dl_all:
        name = dl.find("dt").text.strip()
        address = dl.find("dd").text.strip()
        gu = address.split(" ")[1]
        each = {
            "매장이름":name, "주소":address, "구":gu, "브랜드":"이디야"
        }
        eidya_data.append(each)

In [7]:
len(eidya_data)

1160

In [9]:
df_ediya = pd.DataFrame(eidya_data)
df_ediya.tail()

,Unnamed: 0,매장이름,주소,구,브랜드,위도,경도
1155,1155,용산원효점,"서울 용산구 원효로 70 (원효로4가, 신정빌딩)",용산구,이디야,NaN,NaN
1156,1156,용산중앙점,서울 용산구 한강대로 52-1 (한강로3가),용산구,이디야,NaN,NaN
1157,1157,용산청파점,"서울 용산구 청파로71길 10 (청파동1가, 새마을금고)",용산구,이디야,NaN,NaN
1158,1158,용산효창점,서울 용산구 효창원로66길 2 (효창동),용산구,이디야,NaN,NaN
1159,1159,원효로점,서울 용산구 원효로 257 (원효로1가),용산구,이디야,NaN,NaN


In [13]:
df_coffee = pd.concat([df_starbucks, df_ediya])
# df_coffee =pd.read_csv("./ediya_starbucks_coffe_maps.csv", encoding="utf-8", index_col=0)
# df_coffee.reset_index(drop=True, inplace=True)

In [54]:
df_coffee.tail()

,매장이름,주소,구,브랜드,위도,경도
1307,용산중앙점,서울 용산구 한강대로 52-1 (한강로3가),용산구,이디야,37.525589,126.964826
1308,용산청파점,"서울 용산구 청파로71길 10 (청파동1가, 새마을금고)",용산구,이디야,37.544676,126.970027
1309,용산효창점,서울 용산구 효창원로66길 2 (효창동),용산구,이디야,37.541042,126.963486
1310,용산후암점,"서울 용산구 후암로 40-2 (후암동, 성원빌딩)",용산구,이디야,37.549481,126.977665
1311,원효로점,서울 용산구 원효로 257 (원효로1가),용산구,이디야,37.539932,126.967461


In [66]:
df_coffee.to_csv("~/ediya_starbucks_coffe_maps.csv", sep=",", encoding="utf-8")

In [64]:
df_coffee_csv = pd.read_csv("../ediya_starbucks_coffe_maps.csv", encoding="utf-8", index_col=0)

In [61]:
df_coffee_csv.tail()

,매장이름,주소,구,브랜드,위도,경도
1307,용산중앙점,서울 용산구 한강대로 52-1 (한강로3가),용산구,이디야,37.525589,126.964826
1308,용산청파점,"서울 용산구 청파로71길 10 (청파동1가, 새마을금고)",용산구,이디야,37.544676,126.970027
1309,용산효창점,서울 용산구 효창원로66길 2 (효창동),용산구,이디야,37.541042,126.963486
1310,용산후암점,"서울 용산구 후암로 40-2 (후암동, 성원빌딩)",용산구,이디야,37.549481,126.977665
1311,원효로점,서울 용산구 원효로 257 (원효로1가),용산구,이디야,37.539932,126.967461


In [68]:
df_coffee_csv.tail(2)

,매장이름,주소,구,브랜드,위도,경도
1310,용산후암점,"서울 용산구 후암로 40-2 (후암동, 성원빌딩)",용산구,이디야,37.549481,126.977665
1311,원효로점,서울 용산구 원효로 257 (원효로1가),용산구,이디야,37.539932,126.967461


In [69]:
df_coffee_csv["값"] = 1
df_coffee_csv.tail()

,매장이름,주소,구,브랜드,위도,경도,값
1307,용산중앙점,서울 용산구 한강대로 52-1 (한강로3가),용산구,이디야,37.525589,126.964826,1
1308,용산청파점,"서울 용산구 청파로71길 10 (청파동1가, 새마을금고)",용산구,이디야,37.544676,126.970027,1
1309,용산효창점,서울 용산구 효창원로66길 2 (효창동),용산구,이디야,37.541042,126.963486,1
1310,용산후암점,"서울 용산구 후암로 40-2 (후암동, 성원빌딩)",용산구,이디야,37.549481,126.977665,1
1311,원효로점,서울 용산구 원효로 257 (원효로1가),용산구,이디야,37.539932,126.967461,1
